# Notebook to explore the data for the classifier

In [0]:
input_container_path = f"wasbs://trainingsamples@challengebasf.blob.core.windows.net"
input_blob_folder = f"{input_container_path}/file_names/"
target_files = spark.read.parquet(input_blob_folder)
target_file_p = target_files.toPandas()
target_file_names = target_file_p["_file"].tolist()

/databricks/spark/python/pyspark/sql/types.py:1636: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
 return pa.schema(fields)

In [0]:
from pyspark.sql import functions as sf
input_text_data = f"wasbs://processedtextdata@challengebasf.blob.core.windows.net/output_data/"
text = spark.read.parquet(input_text_data)
text = text.filter(sf.col("_file").isin(target_file_names))

input_full_data = f"wasbs://filtereddata@challengebasf.blob.core.windows.net/output_data/"
full = spark.read.parquet(input_full_data)
full = full.filter(sf.col("_file").isin(target_file_names))

## Feature selection of non-text fields
Data about classification-ipc could be useful for the classifier. One patent can have multiple classifications, so we are going to create a specific functions to emulate the result of a OneHotEncoder over the set of sections and sections and class. 
Another subcategories are not included to avoid increase the dimensionality in huge numbers.

In [0]:
sections_ipcr = ["A", "B", "C", "D", "E", "F", "G"]
sections_class_ipcr = [
    "A01", "A21", "A22", "A23", "A24", "A41", "A42", "A43", "A44", "A45", "A46", "A47", "A61", "A62", "A63", "A99",
    "B01", "B02", "B03", "B04", "B05", "B06", "B07", "B08", "B09",
    "B21", "B22", "B23", "B24", "B25", "B26", "B27", "B28", "B29", "B30", "B31", "B32",
    "B41", "B42", "B43", "B44", "B60", "B61", "B62", "B63", "B64", "B65", "B66", "B67", "B68", "B81", "B82", "B99",
    "C01", "C02", "C03", "C04", "C05", "C06", "C07", "C08", "C09", "C10", "C11", "C12", "C13", "C14",
    "C21", "C22", "C23", "C25", "C30", "C40", "C99",
    "D01", "D02", "D03", "D04", "D05", "D06", "D07", "D21", "D99",
    "E01", "E02", "E03", "E04", "E05", "E06", "E21", "E99",
    "F01", "F02", "F03", "F04", "F15", "F16", "F17", "F21",
    "F22", "F23", "F24", "F25", "F26", "F27", "F28", "F41", "F42", "F99",
    "G01", "G02", "G03", "G04", "G05", "G06", "G07", "G08", "G09", "G10", "G11", "G12", "G21", "G99",
    "H01", "H02", "H03", "H04", "H05", "H99",
]

from pyspark.sql.types import *
schema = StructType([
    StructField("sections", ArrayType(StringType()), True),
    StructField("sections_class", ArrayType(StringType()), True)
])

In [0]:
@sf.udf(schema)
def extract_ipcr(value):
    """Extract all the sections and section/class of a patent and returns all its possible values for each type"""
    if value is None:
        return [], []
    # Return type has to be a list. Set gives null result in spark
    sections = list()
    sections_class = list()
    for item in value:
        if item is None:
            pass
        else:
            text = item["text"].strip().upper()
            if text is not None and len(text) > 3:
                sections.append(text[0])
                sections_class.append(item["text"][:3])
    # Remove duplicates
    sections = list(set(sections))
    sections_class = list(set(sections_class))
    return sections, sections_class

In [0]:
col = "bibliographic-data_classifications-ipcr_classification-ipcr"
df = full.select("_file", col)
df = df.withColumn("ipcr_values", extract_ipcr(sf.col(col)))
df = df.withColumn("ipcr_sections", sf.col("ipcr_values.sections"))
df = df.withColumn("ipcr_sections_class", sf.col("ipcr_values.sections_class"))
for section in sections_ipcr:
  df = df.withColumn(f"section_{section}", sf.array_contains(sf.col("ipcr_sections"), section).astype(IntegerType()))

for section_class in sections_class_ipcr:
  df = df.withColumn(f"section_class_{section_class}", sf.array_contains(sf.col("ipcr_sections_class"), section_class).astype(IntegerType()))

df = df.drop(col)

In [0]:
df_p = df.toPandas()

In [0]:
df.printSchema()

root
-- _file: string (nullable = true)
-- ipcr_values: struct (nullable = true)
 |-- sections: array (nullable = true)
 | |-- element: string (containsNull = true)
 |-- sections_class: array (nullable = true)
 | |-- element: string (containsNull = true)
-- ipcr_sections: array (nullable = true)
 |-- element: string (containsNull = true)
-- ipcr_sections_class: array (nullable = true)
 |-- element: string (containsNull = true)
-- section_A: integer (nullable = true)
-- section_B: integer (nullable = true)
-- section_C: integer (nullable = true)
-- section_D: integer (nullable = true)
-- section_E: integer (nullable = true)
-- section_F: integer (nullable = true)
-- section_G: integer (nullable = true)
-- section_class_A01: integer (nullable = true)
-- section_class_A21: integer (nullable = true)
-- section_class_A22: integer (nullable = true)
-- section_class_A23: integer (nullable = true)
-- section_class_A24: integer (nullable = true)
-- section_class_A41: integer (nullable = true)
-- section_class_A42: integer (nullable = true)
-- section_class_A43: integer (nullable = true)
-- section_class_A44: integer (nullable = true)
-- section_class_A45: integer (nullable = true)
-- section_class_A46: integer (nullable = true)
-- section_class_A47: integer (nullable = true)
-- section_class_A61: integer (nullable = true)
-- section_class_A62: integer (nullable = true)
-- section_class_A63: integer (nullable = true)
-- section_class_A99: integer (nullable = true)
-- section_class_B01: integer (nullable = true)
-- section_class_B02: integer (nullable = true)
-- section_class_B03: integer (nullable = true)
-- section_class_B04: integer (nullable = true)
-- section_class_B05: integer (nullable = true)
-- section_class_B06: integer (nullable = true)
-- section_class_B07: integer (nullable = true)
-- section_class_B08: integer (nullable = true)
-- section_class_B09: integer (nullable = true)
-- section_class_B21: integer (nullable = true)
-- section_class_B22: integer (nullable = true)
-- section_class_B23: integer (nullable = true)
-- section_class_B24: integer (nullable = true)
-- section_class_B25: integer (nullable = true)
-- section_class_B26: integer (nullable = true)
-- section_class_B27: integer (nullable = true)
-- section_class_B28: integer (nullable = true)
-- section_class_B29: integer (nullable = true)
-- section_class_B30: integer (nullable = true)
-- section_class_B31: integer (nullable = true)
-- section_class_B32: integer (nullable = true)
-- section_class_B41: integer (nullable = true)
-- section_class_B42: integer (nullable = true)
-- section_class_B43: integer (nullable = true)
-- section_class_B44: integer (nullable = true)
-- section_class_B60: integer (nullable = true)
-- section_class_B61: integer (nullable = true)
-- section_class_B62: integer (nullable = true)
-- section_class_B63: integer (nullable = true)
-- section_class_B64: integer (nullable = true)
-- section_class_B65: integer (nullable = true)
-- section_class_B66: integer (nullable = true)
-- section_class_B67: integer (nullable = true)
-- section_class_B68: integer (nullable = true)
-- section_class_B81: integer (nullable = true)
-- section_class_B82: integer (nullable = true)
-- section_class_B99: integer (nullable = true)
-- section_class_C01: integer (nullable = true)
-- section_class_C02: integer (nullable = true)
-- section_class_C03: integer (nullable = true)
-- section_class_C04: integer (nullable = true)
-- section_class_C05: integer (nullable = true)
-- section_class_C06: integer (nullable = true)
-- section_class_C07: integer (nullable = true)
-- section_class_C08: integer (nullable = true)
-- section_class_C09: integer (nullable = true)
-- section_class_C10: integer (nullable = true)
-- section_class_C11: integer (nullable = true)
-- section_class_C12: integer (nullable = true)
-- section_class_C13: integer (nullable = true)
-- section_class_C14: integer (nullable = true)
-- section_class_C21: integer (nullable = true)
-- section_class_C22: int

In [0]:
import pandas as pd
total = pd.merge(df_p, target_file_p, on=["_file"], how="outer", indicator=True)
assert total[total["_merge"] != "both"].empty

In [0]:
cols = ["section_A", "section_B", "section_C", "section_D", "section_E", "section_F", "section_G"]
positives = total[total["positive_data"] == 1][cols]
negatives = total[total["positive_data"] == 0][cols]
num_pos = positives.shape[0]
num_neg = negatives.shape[0]
total_pos = positives.sum()
total_neg = negatives.sum()
perc_pos = 100 * total_pos / num_pos
perc_neg = 100 * total_neg / num_neg
result = pd.concat([perc_pos, perc_neg, total_pos, total_neg], axis=1)
result.columns = ["perc_positives", "perc_negatives", "total_positives", "total_negatives"]

In [0]:
result

,perc_positives,perc_negatives,total_positives,total_negatives
section_A,11.864407,31.114809,7,187
section_B,8.474576,16.139767,5,97
section_C,5.084746,9.484193,3,57
section_D,0.000000,1.663894,0,10
section_E,3.389831,4.159734,2,25
section_F,22.033898,14.975042,13,90
section_G,44.067797,23.128120,26,139


In [0]:
display(result)

perc_positives,perc_negatives,total_positives,total_negatives
11.864406779661017,31.114808652246257,7,187
8.474576271186441,16.139767054908486,5,97
5.084745762711864,9.484193011647255,3,57
0.0,1.663893510815308,0,10
3.389830508474576,4.159733777038269,2,25
22.033898305084747,14.97504159733777,13,90
44.067796610169495,23.12811980033278,26,139


The variables shows that energy consumption patents have more probability than the average for section G and less for section A.

In [0]:
cols = ['section_class_A01',
        'section_class_A21', 'section_class_A22', 'section_class_A23', 'section_class_A24', 'section_class_A41',
        'section_class_A42', 'section_class_A43', 'section_class_A44', 'section_class_A45', 'section_class_A46',
        'section_class_A47', 'section_class_A61', 'section_class_A62', 'section_class_A63', 'section_class_A99',
        'section_class_B01', 'section_class_B02', 'section_class_B03', 'section_class_B04', 'section_class_B05',
        'section_class_B06', 'section_class_B07', 'section_class_B08', 'section_class_B09', 'section_class_B21',
        'section_class_B22', 'section_class_B23', 'section_class_B24', 'section_class_B25', 'section_class_B26',
        'section_class_B27', 'section_class_B28', 'section_class_B29', 'section_class_B30', 'section_class_B31',
        'section_class_B32', 'section_class_B41', 'section_class_B42', 'section_class_B43', 'section_class_B44',
        'section_class_B60', 'section_class_B61', 'section_class_B62', 'section_class_B63', 'section_class_B64',
        'section_class_B65', 'section_class_B66', 'section_class_B67', 'section_class_B68', 'section_class_B81',
        'section_class_B82', 'section_class_B99', 'section_class_C01', 'section_class_C02', 'section_class_C03',
        'section_class_C04', 'section_class_C05', 'section_class_C06', 'section_class_C07', 'section_class_C08',
        'section_class_C09', 'section_class_C10', 'section_class_C11', 'section_class_C12', 'section_class_C13',
        'section_class_C14', 'section_class_C21', 'section_class_C22', 'section_class_C23', 'section_class_C25',
        'section_class_C30', 'section_class_C40', 'section_class_C99', 'section_class_D01', 'section_class_D02',
        'section_class_D03', 'section_class_D04', 'section_class_D05', 'section_class_D06', 'section_class_D07',
        'section_class_D21', 'section_class_D99', 'section_class_E01', 'section_class_E02', 'section_class_E03',
        'section_class_E04', 'section_class_E05', 'section_class_E06', 'section_class_E21', 'section_class_E99',
        'section_class_F01', 'section_class_F02', 'section_class_F03', 'section_class_F04', 'section_class_F15',
        'section_class_F16', 'section_class_F17', 'section_class_F21', 'section_class_F22', 'section_class_F23',
        'section_class_F24', 'section_class_F25', 'section_class_F26', 'section_class_F27', 'section_class_F28',
        'section_class_F41', 'section_class_F42', 'section_class_F99', 'section_class_G01', 'section_class_G02',
        'section_class_G03', 'section_class_G04', 'section_class_G05', 'section_class_G06', 'section_class_G07',
        'section_class_G08', 'section_class_G09', 'section_class_G10', 'section_class_G11', 'section_class_G12',
        'section_class_G21', 'section_class_G99', 'section_class_H01', 'section_class_H02', 'section_class_H03',
        'section_class_H04', 'section_class_H05', 'section_class_H99',
        ]
positives = total[total["positive_data"] == 1][cols]
negatives = total[total["positive_data"] == 0][cols]
num_pos = positives.shape[0]
num_neg = negatives.shape[0]
total_pos = positives.sum()
total_neg = negatives.sum()
perc_pos = 100 * total_pos / num_pos
perc_neg = 100 * total_neg / num_neg
section_class = pd.concat([perc_pos, perc_neg, total_pos, total_neg], axis=1)
section_class.columns = ["perc_positives", "perc_negatives", "total_positives", "total_negatives"]

In [0]:
section_class

,perc_positives,perc_negatives,total_positives,total_negatives
section_class_A01,0.000000,3.993344,0,24
section_class_A21,0.000000,0.332779,0,2
section_class_A22,0.000000,0.000000,0,0
section_class_A23,1.694915,0.998336,1,6
section_class_A24,0.000000,0.499168,0,3
section_class_A41,0.000000,0.831947,0,5
section_class_A42,0.000000,0.000000,0,0
section_class_A43,0.000000,0.332779,0,2
section_class_A44,0.000000,0.332779,0,2
section_class_A45,5.084746,1.331115,3,8


In [0]:
display(section_class)

perc_positives,perc_negatives,total_positives,total_negatives
0.0,3.9933444259567388,0,24
0.0,0.33277870216306155,0,2
0.0,0.0,0,0
1.694915254237288,0.9983361064891847,1,6
0.0,0.49916805324459235,0,3
0.0,0.831946755407654,0,5
0.0,0.0,0,0
0.0,0.33277870216306155,0,2
0.0,0.33277870216306155,0,2
5.084745762711864,1.3311148086522462,3,8


## Feature selection of text fields
We have processed the full text of patents. In this point we are going to extract values from specific text fields.  
¿Patents with energy in its title or claims are more probable to deal with energy consumption?  
¿Are there some specific bi-grams?

In [0]:
text.printSchema()

root
-- _file: string (nullable = true)
-- english_text: string (nullable = true)
-- abstract_text: string (nullable = true)
-- title_text: string (nullable = true)
-- claims_text: string (nullable = true)
-- english_text_features: array (nullable = true)
 |-- element: string (containsNull = true)
-- english_text_stopwords: array (nullable = true)
 |-- element: string (containsNull = true)
-- abstract_text_features: array (nullable = true)
 |-- element: string (containsNull = true)
-- abstract_text_stopwords: array (nullable = true)
 |-- element: string (containsNull = true)
-- title_text_features: array (nullable = true)
 |-- element: string (containsNull = true)
-- title_text_stopwords: array (nullable = true)
 |-- element: string (containsNull = true)
-- claims_text_features: array (nullable = true)
 |-- element: string (containsNull = true)
-- claims_text_stopwords: array (nullable = true)
 |-- element: string (containsNull = true)

###Energy in text, claims or abstract

In [0]:
text = text.withColumn("flag_energy_title", sf.array_contains("title_text_features", "energy"))
text = text.withColumn("flag_energy_abstract", sf.array_contains("abstract_text_features", "energy"))
text = text.withColumn("flag_energy_claims", sf.array_contains("claims_text_features", "energy"))
text_p = text.select("_file", "title_text_features", "abstract_text_features", "claims_text_features", "flag_energy_title", "flag_energy_abstract", "flag_energy_claims").toPandas()

In [0]:
import pandas as pd
text_p = pd.merge(text_p, target_file_p, on=["_file"], how="outer", indicator=True)
assert text_p[text_p["_merge"] != "both"].empty

In [0]:
to_int_cols = ["flag_energy_title", "flag_energy_abstract", "flag_energy_claims"]
for col in to_int_cols:
  text_p.loc[:, col] = text_p[col].astype(int)

In [0]:
text_p.head()

,_file,title_text_features,abstract_text_features,claims_text_features,flag_energy_title,flag_energy_abstract,flag_energy_claims,positive_data,_merge
0,CN101939562A.xml,"[reduction, gear]","[reduction, gear, reduce, dimension, particula...","[one, reducer, within, house, include, outer, ...",0,0,0,0,both
1,CN103368425A.xml,"[synchronous, rectification, drive, circuit]","[invention, disclose, synchronous, rectificati...","[synchronous, rectify, drive, circuit, charact...",0,0,0,0,both
2,CN101600393A.xml,"[ultrasonic, device, cut, coagulate]","[ultrasonic, surgical, instrument, configure, ...","[ultrasonic, surgical, instrument, comprise, s...",0,0,1,0,both
3,CN102555430A.xml,"[electrothermal, transfer, device, electrother...","[invention, relate, electrothermal, transfer, ...","[electrothermal, transfer, device, comprise, s...",0,0,0,0,both
4,CN103230758A.xml,"[magnetic, stir, actuator, monitorable, rotate...","[invention, disclose, magnetic, stir, actuator...","[one, variablerotation, monitor, magnetic, sti...",0,0,0,0,both


In [0]:
for col in to_int_cols:
  result_p = text_p.groupby("_file").agg({"positive_data": "min", col: "max"})
  positives = result_p[result_p["positive_data"] == 1]
  negatives = result_p[result_p["positive_data"] == 0]
  print(f'Percentage of positive data in {col}: {100 * positives[col].sum() / positives.shape[0]}')
  print(f'Percentaje of negative data in {col}: {100 * negatives[col].sum() / negatives.shape[0]}')

Percentage of positive data in flag_energy_title: 52.54237288135593
Percentaje of negative data in flag_energy_title: 0.49916805324459235
Percentage of positive data in flag_energy_abstract: 44.067796610169495
Percentaje of negative data in flag_energy_abstract: 3.327787021630616
Percentage of positive data in flag_energy_claims: 44.067796610169495
Percentaje of negative data in flag_energy_claims: 4.3261231281198

It is clear that patents with energy in its title are more probable to be about energy consumption

Future work: 
- Option 1: analyze words and bigram distributions between postive and negative and create new manual features
- Option 2: Apply a count vectorizer/idf to each text column (title, abstract, claims, description) independenly and merge the result to create the input to the classifier (check dimensionality)